In [48]:
import datetime

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession

In [ ]:
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [2]:
# import pandas as pd
# import datetime

# # import data
# data = pd.read_csv("/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")

# # convert day column type to datetime
# data['day'] = pd.to_datetime( data['day'], infer_datetime_format=True, yearfirst=True)

# # create new column
# data['date_timestamp'] = pd.to_datetime(data.day.astype(str) + ' ' + data.time)

# # Drop old columns
# data.drop(['day','time'],inplace=True,axis=1)

# # Write to csv
# data.to_csv("/home/bluepi/Downloads/Update/product_info/main table/main table.csv", index=False)

In [3]:
mainTable = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main table.csv")

In [4]:
mainTable.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- date_timestamp: timestamp (nullable = true)



In [36]:
from pyspark.sql.functions import year, month, dayofyear, hour, minute, second

mainTable.select([year(mainTable['date_timestamp']).alias("Year"),
                  month(mainTable['date_timestamp']).alias("Month"),
                  dayofyear(mainTable['date_timestamp']).alias("DayOfYear"),
                  hour(mainTable['date_timestamp']).alias("Hour"),
                  minute(mainTable['date_timestamp']).alias("Minute"),
                  second(mainTable['date_timestamp']).alias("Second")]).show(3)

+----+-----+---------+----+------+------+
|Year|Month|DayOfYear|Hour|Minute|Second|
+----+-----+---------+----+------+------+
|2020|    9|      247|  19|    24|    25|
|2020|    3|       78|  16|    41|    43|
|2020|    8|      216|  22|    17|     5|
+----+-----+---------+----+------+------+
only showing top 3 rows

None


In [47]:
mainTable.select('date_timestamp').orderBy(mainTable.date_timestamp.asc()).show(20)

+-------------------+
|     date_timestamp|
+-------------------+
|2020-01-03 04:07:54|
|2020-01-03 04:38:21|
|2020-01-03 04:41:13|
|2020-01-03 05:24:56|
|2020-01-03 22:27:04|
|2020-01-03 23:54:49|
|2020-02-03 04:05:43|
|2020-02-03 04:35:29|
|2020-02-03 05:02:57|
|2020-02-03 06:12:33|
|2020-02-03 10:30:17|
|2020-02-03 10:58:29|
|2020-02-03 14:36:42|
|2020-02-03 16:38:45|
|2020-02-03 17:03:12|
|2020-02-03 18:43:16|
|2020-02-03 23:43:02|
|2020-03-03 03:01:52|
|2020-03-03 04:20:22|
|2020-03-03 04:30:40|
+-------------------+
only showing top 20 rows



In [74]:
# Address to the product_info folder
address = "/home/bluepi/Downloads/Update/product_info/"
previous_day = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')

# Address to the previous day folder
new_address = address + previous_day

# # read the previous day folder
data = spark.read.format('csv') \
          .options( header=True, inferschema=True ) \
          .load(new_address)

In [75]:
data.show(3)

+----+--------+-----+-------------------+-----------+
|p_id|  p_name|price|     Date_timestamp|record_type|
+----+--------+-----+-------------------+-----------+
| 179|Treeflex|  145|2020-03-24 22:05:51|          U|
| 848| Regrant|  655|2020-03-24 21:55:48|          U|
| 993| Fixflex|   21|2020-03-24 22:38:09|          U|
+----+--------+-----+-------------------+-----------+
only showing top 3 rows



In [76]:
data.orderBy(data.Date_timestamp.asc()).show(40)

+----+-----------+-----+-------------------+-----------+
|p_id|     p_name|price|     Date_timestamp|record_type|
+----+-----------+-----+-------------------+-----------+
| 833|     Hatity|  277|2020-03-24 03:15:21|          D|
| 855|    Veribet|  597|2020-03-24 03:17:59|          U|
| 524|     Keylex|  264|2020-03-24 03:28:37|          D|
| 153|    Quo Lux|  480|2020-03-24 03:34:15|          U|
| 456|       Stim| 1302|2020-03-24 03:46:06|          U|
| 251|         It| 1089|2020-03-24 03:48:56|          D|
| 369|    Fintone|  525|2020-03-24 04:08:17|          U|
| 524|       Stim| 1437|2020-03-24 04:09:53|          D|
| 824|   Aerified| 1356|2020-03-24 04:22:22|          U|
| 308|    Konklab|  387|2020-03-24 04:26:49|          U|
| 251|    Zontrax|    7|2020-03-24 04:29:59|          D|
| 851|      Alpha| 1019|2020-03-24 09:21:26|          U|
| 853|    Konklab| 1315|2020-03-24 09:30:56|          U|
| 669|    Sonsing|  309|2020-03-24 09:32:12|          U|
| 466|    Fix San|  295|2020-03